# Evaluation of SIRET index on French RESTAURANTS open database

In [26]:
%reload_ext autoreload
%autoreload 2

In [27]:
from worksites import WorkSiteName, GeoWorkSiteName, geocode_worksites
from queries import request_elastic, get_answer
import pandas as pd
import os
import elasticsearch as es

In [28]:
try:
    os.remove("export_alimconfiance.csv")
except:
    pass
!wget -c "https://www.data.gouv.fr/fr/datasets/r/fff0cc27-977b-40d5-9c11-f7e4e79a0b72" -O export_alimconfiance.csv

--2022-12-03 16:07:05--  https://www.data.gouv.fr/fr/datasets/r/fff0cc27-977b-40d5-9c11-f7e4e79a0b72
Resolving www.data.gouv.fr (www.data.gouv.fr)... 37.59.183.93
Connecting to www.data.gouv.fr (www.data.gouv.fr)|37.59.183.93|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://dgal.opendatasoft.com/explore/dataset/export_alimconfiance/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true [following]
--2022-12-03 16:07:05--  https://dgal.opendatasoft.com/explore/dataset/export_alimconfiance/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true
Resolving dgal.opendatasoft.com (dgal.opendatasoft.com)... 34.249.199.226, 34.248.20.69
Connecting to dgal.opendatasoft.com (dgal.opendatasoft.com)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘export_alimconfiance.csv’

export_alimconfianc     [             <=>    ]   6.44M   393KB/s    in 19s  

In [29]:
# Preparing evaluatest dataset (database of French restaurants)

In [30]:
size_test = 10
df = pd.read_csv("export_alimconfiance.csv", sep=";", dtype=str)[["APP_Libelle_etablissement", "SIRET", "Libelle_commune", "Code_postal", "Adresse_2_UA", "geores"]]

In [31]:
df = df[(df["Code_postal"].str[:2] != "95") & (df["Code_postal"].str[:2] != "94")]

In [32]:
wks = [
    WorkSiteName(**{
        "siret":row["SIRET"],
        "name": row["APP_Libelle_etablissement"],
        "postcode": row["Code_postal"],
        "cityname": row["Libelle_commune"], 
        "address": row["Adresse_2_UA"]
    })
    for k, row in df.head(size_test).iterrows()
]

In [33]:
geowks = geocode_worksites(wks)

In [34]:
ELASTIC_URL = "http://elasticsearch-master:9200"
conn = es.Elasticsearch(ELASTIC_URL)

In [35]:
# Check if the first response of Elasticsearch

In [37]:
geo_threshold = 0.7

In [38]:

score = 0
for geowk in geowks:
    answer = get_answer(conn, geoworksite=geowk, siret_truth=geowk.siret, geo_threshold=geo_threshold)
    print("----")
    print(geowk.name)
    if answer:
        score += 1
        print("{} - {} - {}".format(answer["_score"], geowk.siret, answer["_source"]["nom_complet"]))
    else:
        print("no match")

print(f"\n\nSCORE OF RETRIEVAL: {score/size_test*100}%")

/opt/mamba/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


----
LE SAINT JOSEPH (RESTAURANT DE LA BOULANGERIE)
48.913036 - 31017429700013 - restaurant de la boulangerie
----
NATURALIA (NATURALIA FRANCE)
73.55325 - 30247464801894 - naturalia france
----
SOCIETE CAPOCCI
75.06476 - 31544532000031 - societe capocci
----
SOCIETE BISCUITERIE ERTE
87.90961 - 30918204600024 - societe biscuiterie erte
----
LECLERC (PLOUDAL DISTRIBUTION)
65.48759 - 30959801900052 - ploudal distribution
----
AUX PRODUITS SAVOYARDS (SICA L'ARVAN)
73.71382 - 31739832900019 - sica l'arvan
----
INTERMARCHE (MADIF)
57.327126 - 32676801700021 - madif
----
ETABLISSEMENTS MICHEL COME
63.328156 - 31773315200019 - etablissements michel come
----
SICA BEAUFORT DES MONTAGNES DE MOUTIERS
82.718056 - 32020196500011 - sica beaufort des montagnes de moutiers
----
LIDL (LIDL)
40.758163 - 34326262223612 - lidl snc lidl


SCORE OF RETRIEVAL: 100.0%


In [25]:
df

,APP_Libelle_etablissement,SIRET,Libelle_commune,Code_postal,Adresse_2_UA,geores
1,LE SAINT JOSEPH (RESTAURANT DE LA BOULANGERIE),31017429700013,LYON,69002,46 R MERCIERE,"45.762926,4.832648"
3,NATURALIA (NATURALIA FRANCE),30247464801894,PUTEAUX,92800,36 RUE EUGENE EICHENBERGER,"48.881317,2.23746"
4,SOCIETE CAPOCCI,31544532000031,VERSAILLES,78000,CARRE A LA VIANDE,"48.797148,2.127159"
5,SOCIETE BISCUITERIE ERTE,30918204600024,LE PERRAY EN YVELINES,78610,LIEUDIT LES HAPHLERIES,"48.703268,1.855354"
6,LECLERC (PLOUDAL DISTRIBUTION),30959801900052,PLOUDALMEZEAU,29830,KERUSCAT,"48.540309,-4.66693"
...,...,...,...,...,...,...
33837,HOTEL DIEU DE PARIS HOPITAL,26750045200599,Paris 4e Arrondissement,75004,6 OFFICES / UNITES DE SOINS,"48.854242,2.351101"
33838,LYCEE GENERAL TECHNOLOGIQUE JULES FERRY,19781845300015,Conflans-Sainte-Honorine,78700,"7, rue Jules Bouyssel","48.998733,2.096114"
33839,LA FA BRICK,83833633700020,Paris 18e Arrondissement,75018,47 RUE ORDENER,"48.890917,2.352527"
33840,LES ATELIERS DU GOUT,31933184900053,Barbezieux-Saint-Hilaire,16300,"6, rte de Montmoreau","45.467673,-0.150544"
